# Exploring coordinate systems and projections in Python

Understanding coordinate systems and projections when working with geospatial data is important to:

- Ensure your data are properly mapped on the Earth's surface  
- Ensure analytical work and results are accurate  

Nowadays, most of the mathematics and conversions are cleanly handled under-the-hood, but the decisions the Data Scientist makes are still important.

This notebook will explore projections and coordinate systems and investigate how changes in projections can affect results


In [ ]:
import sys, os
import urllib.parse
import requests
import rasterio

import matplotlib.pyplot as plt
import pandas as pd 
import geopandas as gpd

from io import BytesIO

In [ ]:
# Choose and Map your favourite country
iso3 = "GHA"

# Define some input variables for data exploration
### Official World Bank Group boundary dataset - https://datacatalog.worldbank.org/search/dataset/0038272/World-Bank-Official-Boundaries
world_adm1_boundaries ="https://datacatalogfiles.worldbank.org/ddh-published-v2/0038272/5/DR0095369/World%20Bank%20Official%20Boundaries%20(GeoJSON)/World%20Bank%20Official%20Boundaries%20-%20Admin%201.geojson"
adm1_bounds = gpd.read_file(world_adm1_boundaries)

In [ ]:
# We can also read these from the ESRI rest endpoint through the World Bank's GeoWB
queryable_geowb_url = urllib.parse.quote(f'https://services.arcgis.com/iQ1dY19aHwbSDYIF/arcgis/rest/services/World_Bank_Global_Administrative_Divisions/FeatureServer/2/query?where="ISO_A3"=\'{iso3}\'&f=pgeojson', safe=':/?=&')
country_bound = gpd.read_file(queryable_geowb_url)

In [ ]:
# SSL errors are common when working inside the World Bank firewall. If you encounter these, you can bypass them with the code below
response = requests.get(queryable_geowb_url, verify=False) # <--- Ignore SSL
country_bound = gpd.read_file(BytesIO(response.content))

In [ ]:
# Plot the country data 3 ways
country_bound.union_all()

In [ ]:
country_bound.plot()

In [ ]:
country_bound.explore().save("country_bound.html")

## How do projections affect visuals

Search for different projections for your country at epsg.io

In [ ]:
projection_1 = 2136  # https://epsg.io/2136
projection_2 = 32630 # https://epsg.io/32630
projection_3 = "ESRI:54009" # https://spatialreference.org/ref/esri/54009/ - this is a world projection that preserves area, but distorts shape and distance. It is often used for global scale mapping.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))

country_bound.to_crs(projection_1).plot(ax=axes[0])
country_bound.to_crs(projection_2).plot(ax=axes[1])
country_bound.to_crs(projection_3).plot(ax=axes[2])

In [ ]:
# Compare area calculations across projections
country_bound["area_projection_1"] = country_bound.to_crs(projection_1).area/1000000 # Convert from m^2 to km^2
country_bound["area_projection_2"] = country_bound.to_crs(projection_2).area/1000000 # Convert from m^2 to km^2
country_bound["area_projection_3"] = country_bound.to_crs(projection_3).area/1000000 # Convert from m^2 to km^2

country_bound.sort_values("area_projection_1", ascending=False)

# Why is the first projection so different?